# 🚀 SmartCash: EfficientNet Training Notebook
Notebook untuk training model deteksi mata uang menggunakan YOLOv5 dengan EfficientNet-B4 backbone
* File: 03_efficientnet_training.ipynb
* Author: Alfrida Sabar
## 🔍 Tahap 1: Impor Pustaka Standar dan Kustom
Setup import untuk modul-modul yang dibutuhkan dalam proses training


In [ ]:
# Setup import untuk modul-modul yang dibutuhkan dalam proses training
import os
import sys
import torch
import numpy as np
import yaml
from pathlib import Path

# Impor modul kustom
from utils.logger import SmartCashLogger
from utils.experiment_tracker import ExperimentTracker
from handlers.data_handler import DataHandler
from handlers.roboflow_handler import RoboflowHandler
from handlers.efficient_handler import EfficientNetHandler
from utils.preprocessing import ImagePreprocessor

# Setup logger
logger = SmartCashLogger(__name__)

## 🗂️ Tahap 2: Konfigurasi Path dan Lingkungan
Menyiapkan direktori dan path yang diperlukan untuk proses training
### 📍 Definisi Lokasi Direktori Utama

In [ ]:
BASE_DIR = Path.cwd()
CONFIG_PATH = BASE_DIR / 'configs' / 'base_config.yaml'
DATA_DIR = BASE_DIR / 'data'
RESULTS_DIR = BASE_DIR / 'results'

# Buat direktori jika belum ada
RESULTS_DIR.mkdir(exist_ok=True)


## 📄 Tahap 3: Pemuatan Konfigurasi
Memuat konfigurasi dari file YAML untuk kontrol proses training
### 🔧 Baca Konfigurasi Eksperimen

In [ ]:
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

## 🧹 Tahap 4: Persiapan Data
Melakukan preprocessing dan validasi dataset untuk training
### 📊 Preprocessing Dataset

In [ ]:
# Setup data handler
data_handler = DataHandler(CONFIG_PATH)
data_handler.setup_dataset_structure()

# Preprocessing
preprocessor = ImagePreprocessor(CONFIG_PATH)
preprocessor.process_dataset(
    input_dir=DATA_DIR / 'train', 
    output_dir=DATA_DIR / 'processed_train',
    augment=True
)

# Validasi dataset
preprocessing_stats = preprocessor.validate_preprocessing(
    DATA_DIR / 'processed_train'
)

## 📈 Tahap 5: Inisialisasi Pelacakan Eksperimen
Menyiapkan sistem pelacakan dan logging untuk eksperimen training
### 🔬 Setup Experiment Tracker

In [ ]:
experiment_tracker = ExperimentTracker(
    experiment_name='EfficientNet_SmartCash',
    output_dir=str(RESULTS_DIR)
)

## 🤖 Tahap 6: Inisialisasi Model
Membangun arsitektur model YOLOv5 dengan EfficientNet-B4 backbone

### 🏗️ Konstruksi Model

In [ ]:
efficient_handler = EfficientNetHandler(
    config_path=str(CONFIG_PATH),
    num_classes=len(config['dataset']['classes'])
)

# Build model
efficient_handler.build()

## 🚂 Tahap 7: Training Pipeline
Implementasi keseluruhan proses training dengan mekanisme kontrol yang komprehensif

### 🔄 Definisi Fungsi Training Utama

In [ ]:
def run_notebook():
    """Jalankan keseluruhan training pipeline"""
    try:
        # Start eksperimen
        experiment_tracker.start_run(
            run_name='EfficientNet_Scenario3',
            config=config
        )
        
        # Persiapan data
        train_loader = torch.utils.data.DataLoader(
            # Dataset loading sesuai konfigurasi
            batch_size=config['model']['batch_size'],
            shuffle=True,
            num_workers=config['model']['workers']
        )
        
        val_loader = torch.utils.data.DataLoader(
            # Dataset validation loading
            batch_size=config['model']['batch_size'],
            shuffle=False,
            num_workers=config['model']['workers']
        )
        
        # Training loop
        for epoch in range(config['training']['epochs']):
            logger.start(f"🚀 Epoch {epoch+1}/{config['training']['epochs']}")
            
            # Training
            train_metrics = efficient_handler.train_epoch(train_loader, epoch)
            
            # Validasi
            val_metrics = efficient_handler.validate(val_loader)
            
            # Gabungkan metrics
            combined_metrics = {**train_metrics, **val_metrics}
            
            # Log metrics
            experiment_tracker.log_metrics(combined_metrics, step=epoch)
            
            # Checkpoint
            efficient_handler.save_checkpoint(combined_metrics)
            
            # Early stopping check
            if efficient_handler.early_stopping(val_metrics):
                logger.warning("🛑 Early stopping triggered!")
                break
        
        # Akhiri eksperimen
        experiment_tracker.end_run()
        
        # Visualisasi
        experiment_tracker.plot_metrics()
        
    except Exception as e:
        logger.error(f"❌ Training gagal: {str(e)}")
        experiment_tracker.end_run(status='failed')

## 🏁 Tahap 8: Eksekusi Training
Memulai proses training dan menjalankan keseluruhan pipeline

### 🚀 Mulai Training


In [ ]:
if __name__ == '__main__':
    run_notebook()